# UNION 3D MODEL

## Function Definitions

In [0]:
import tensorflow as tf
import numpy as np
import os
import nibabel as nib
import cv2 as cv
import matplotlib.pyplot as plt
    
def modality(Path,index):
    X = []
    X_per_paitent = []
    p=os.listdir(Path) 
    recs_in=[]
    counter=0
    counter_2=0
    #recs_segmented=[]
    for i in p[:14]:                                                                      # Loading all the folders in the given path
        q = os.listdir(os.path.join(Path,i))     

        x = nib.load(os.path.join(Path,i,q[index]))         
        f = x.get_fdata()
        f = np.asarray(f,'float32')
        
        ct=0
        recs_in.append(f.shape[2])
        #print(counter_2)
        counter_2+=1
        for j in range(f.shape[2]):                                                        # Processing the MRI Scan in the axial view
            _slice = cv.resize(f[:,:,j],(256,256),interpolation=cv.INTER_NEAREST)             # Resizing the slice to the shape(256,256)
            if(index not in [3,4,5,6,7,8,9] and np.sum(_slice) != 0 ): 
                if index==1:
                  ct+=1 
                  counter+=1 
                 # print(counter)                                      # To check whether the slice is null or not
              #  _slice = _slice / (np.max(_slice) + 0.00001)                               # Normalization
                  _slice = (_slice - np.mean(_slice) + 0.00001) / (np.std(_slice) + 0.00001)
                else:
                                              # To check whether the slice is null or not
              #  _slice = _slice / (np.max(_slice) + 0.00001)                               # Normalization
                  _slice = (_slice - np.mean(_slice) + 0.00001) / (np.std(_slice) + 0.00001) # Standardization
            elif(index in [3,4,5,6,7,8,9]):   # if index = 3, Then it is output mask and we don't normalize or standardize it 
                _slice = np.array(_slice)
                _slice[_slice > 0] = 1.0
                _slice[_slice < 0] = 0.0
            _slice = _slice.T
            _slice = _slice[:,:,np.newaxis]
            X.append(_slice)
       
   # X=np.array(X,dtype='float32')
    return X,recs_in

In [0]:
def remove_null_samples(X_Dp, X_Flair, X_Gado, X_T1, X_T2, Y_Manual,recs): 
     
    X=[]
    Y=[]
    counter=0
    counter_2=0
    mult=0;
    count=0
    rec=[]
    keep_count=[]
    keep=[]
    print(recs)
    r=np.array(recs,dtype='float32')
    print(np.sum(r))
    print(len(X_Dp))

    for i in range(len(X_Dp)):  
        if counter==(recs[mult]):
          print(counter)
          mult+=1
          rec.append(count)
          counter=0
          print(counter_2)
          count=0
        final_slice = np.concatenate((X_Dp[i],X_Flair[i],X_Gado[i],X_T1[i],X_T2[i]), axis = -1)
        if(np.sum(final_slice) != 0):        # checking whether the final slice is empty or not             
            X.append(final_slice)
            Y.append(Y_Manual[i])
            
            count+=1
        counter+=1
        counter_2+=1

    
    rec.append(count)
#   Converting the list into array  
    X=np.array(X,dtype='float32')
    Y=np.array(Y,dtype='float32')
    rec=np.array(rec,dtype='float32')
    
    return X,Y,rec


In [0]:
def store_data(X,Y,rec):
    np.save("drive/My Drive/MS_data/X_union_new.npy",X)
    np.save("drive/My Drive/MS_data/Y_union_new.npy",Y)
    np.save("drive/My Drive/MS_data/union_new_rec_after_removal.npy",rec)



In [0]:
import math
def union(Y_1,Y_2,Y_3,Y_4,Y_5,Y_6,Y_7):
  Y=[]
  sum2=[]
  flag=0
  #y=np.array()
  print("A")
  for i in range (len(Y_1)):
    #print(Y_1[i])
    
        f=np.concatenate((Y_1[i],Y_2[i],Y_3[i],Y_4[i],Y_5[i],Y_6[i],Y_7[i]),axis=-1)
        sum=np.sum(f,axis=2)
       # print(sum)
        
          #print(j)
        sum_1=np.divide(sum,7)
        sum_1=np.ceil(sum_1)
        sum2.append(sum_1)

    #sum2=np.array(sum2,dtype='float32')
    
        
  return sum2

## Preprocessing DataSets

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

#import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input ,BatchNormalization , Activation 
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


#import dataPrepare as process
# Loading all the 5 different modalities of each MRI Scan of all 15 different patients and 1st rater Manual SegmentationX_Dp      =   modality(Path,0)
#import Modified_UNet 
#import plots
#import Metrics

# Setting the path
Path='drive/My Drive/Pre-processed'



# Loading all the 5 different modalities of each MRI Scan of all 15 different patients and 1st rater Manual Segmentation
X_Dp_t,rec      =   modality(Path,0)
X_Flair_t,rec_1   =   modality(Path,1)
X_Gado_t,rec    =   modality(Path,2)
X_T1_t,rec      =   modality(Path,10)
X_T2_t,rec      =   modality(Path,11)
rec=np.array(rec_1,dtype='float32')
np.save("drive/My Drive/MS_data/union_new_rec_before_removal.npy",rec)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

#import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input ,BatchNormalization , Activation 
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


#import dataPrepare as process
# Loading all the 5 different modalities of each MRI Scan of all 15 different patients and 1st rater Manual SegmentationX_Dp      =   modality(Path,0)
#import Modified_UNet 
#import plots
#import Metrics

# Setting the path
Path='drive/My Drive/Pre-processed'
Y_1,rec  =   modality(Path,3)
Y_2,rec  =   modality(Path,4)
Y_3,rec  =   modality(Path,5)
Y_4,rec  =   modality(Path,6)
Y_5,rec  =   modality(Path,7)
Y_6,rec  =   modality(Path,8)
Y_7,rec  =   modality(Path,9)


In [0]:
Y_Manual=union(Y_1,Y_2,Y_3,Y_4,Y_5,Y_6,Y_7)

A


In [0]:
import numpy as np

np.save("drive/My Drive/MS_data/Y_manual_new.npy",Y_Manual)

In [0]:
Y_manual=list(Y_Manual)

In [0]:
Y_Manual=np.load("drive/My Drive/MS_data/Y_manual_new.npy")

In [0]:
X, Y,rec = remove_null_samples(X_Dp_t, X_Flair_t, X_Gado_t, X_T1_t, X_T2_t, Y_manual,rec_1)

In [0]:
store_data(X,Y,rec)

In [0]:
import numpy as np
Y=np.load("drive/My Drive/MS_data/Y_union_new.npy")

In [0]:
Y_Manual=list(Y)

In [0]:
Y_1=Y_Manual[:1401]
Y_2=Y_Manual[1401:]

In [0]:
p=[]
y=[]
z=[]
a=[]
n=1
for i in Y_1:
  print(n)
  for j in i:
    for k in j:
      #print(k)
      a.append(k)
     # print(a)
      z.append(a)
      a=[]
    #print(z)
    y.append(z)
    z=[]
  p.append(y)
  y=[]
  n+=1


#print(p)

      

In [0]:
temp=np.array(p)
np.save("drive/My Drive/MS_data/temp_1_new.npy",temp)

In [0]:
q=[]
y=[]
z=[]
a=[]
n=1401
for i in Y_2:
  print(n)
  for j in i:
    for k in j:
      #print(k)
      a.append(k)
     # print(a)
      z.append(a)
      a=[]
    #print(z)
    y.append(z)
    z=[]
  q.append(y)
  y=[]
  n+=1


In [0]:
temp2=np.array(q)
np.save("drive/My Drive/MS_data/temp_2_new.npy",temp2)

In [0]:
temp=np.load("drive/My Drive/MS_data/temp_1_new.npy")
p=list(temp)

In [0]:
temp2=np.load("drive/My Drive/MS_data/temp_2_new.npy")
q=list(temp2)

In [0]:
j=1401
for i in q:
  print(j)
  p.append(i)
  j+=1

In [0]:
np.save("drive/My Drive/MS_data/Y_Manual_2_new.npy",p)

In [0]:
P=np.load("drive/My Drive/MS_data/Y_Manual_2_new.npy")

In [0]:
Y=P

In [0]:
Y=list(Y)

In [0]:
dummy_y=[]
d_temp_1=[]
d_temp_2=[]
for i in range(len(Y[0])):
  d_temp_1=[]
  for j in range(len(Y[0][0])):
    d_temp_2=[]
    for k in range(len(Y[0][0][0])):
      d_temp_2.append(0.)
    d_temp_1.append(d_temp_2)
  dummy_y.append(d_temp_1)
Y_dummy=np.array(dummy_y, dtype='float32')



 



In [0]:
X=np.load("drive/My Drive/MS_data/X_union_new.npy")

In [0]:
X=list(X)

In [0]:
dummy_x=[]
d_temp_1=[]
d_temp_2=[]
for i in range(len(X[0])):
  d_temp_1=[]
  for j in range(len(X[0][0])):
    d_temp_2=[]
    for k in range(len(X[0][0][0])):
      d_temp_2.append(0.)
    d_temp_1.append(d_temp_2)
  dummy_x.append(d_temp_1)
X_dummy=np.array(dummy_x, dtype='float32')

In [0]:
X_dummy

In [0]:
import numpy as np

#Y=np.load("drive/My Drive/MS_data/Y_Manual_2_new.npy")
X=np.load("drive/My Drive/MS_data/X_union_new.npy")
#r=np.load("drive/My Drive/MS_data/union_new_rec_after_removal.npy")

In [0]:
X=list(X)
#Y=list(Y)

In [0]:
Y=Y.tolist()

In [0]:
print(len(X))

2940


In [0]:
extra=[0,1,6,4,6,0,5,0,4,3,1,3,3,0]

In [0]:
ext=np.array(extra,dtype='float32')

In [0]:
s=np.sum(ext)

In [0]:
r=np.load("drive/My Drive/MS_data/union_new_rec_after_removal.npy")

In [0]:
rec=list(r)

In [0]:
rec

In [0]:
j=0
index=0
for i in extra:
  k=0
  
  print(i)
  index=index+rec[j]
  j+=1
  while(k<i):
    index=index+k
    print(index)
    index=int(index)
    Y.insert(index,Y_dummy)
    X.insert(index,X_dummy)

    k+=1

In [0]:
rec=list(r)
X=list(X)
Y=list(Y)

In [0]:
X=np.array(X,dtype='float32')
Y=np.array(Y,dtype='float32')



In [0]:
np.save("drive/My Drive/MS_data/Actual_X.npy",X)
np.save("drive/My Drive/MS_data/Actual_Y.npy",Y)

In [0]:
import numpy as np

Y=np.load("drive/My Drive/MS_data/Actual_X.npy")
X=np.load("drive/My Drive/MS_data/Actual_Y.npy")

In [0]:
X_1=[]
Y_1=[]


In [0]:
X=list(X)
Y=list(Y)

In [0]:
index_x_p=0
length=8
index_x=length
index_y_p=0
index_y=length
x_temp=[]
y_temp=[]




In [0]:
while index_x<=len(X) and index_y<=len(Y):
  x_temp=X[index_x_p:index_x]
  y_temp=Y[index_y_p:index_y]
  x_temp_1=np.array(x_temp,dtype='float32')
  y_temp_1=np.array(y_temp,dtype='float32')
  X_1.append(x_temp_1)
  Y_1.append(y_temp_1)
  index_y_p=index_y_p+length
  index_x_p=index_x_p+length
  index_x=index_x+length
  index_y=index_y+length


In [0]:
X_1=np.array(X_1,dtype='float32')
Y_1=np.array(Y_1,dtype='float32')


In [0]:
np.save("drive/My Drive/MS_data/train_X.npy",X_1)
np.save("drive/My Drive/MS_data/train_Y.npy",Y_1)

In [0]:
import numpy as np

X_1=np.load("drive/My Drive/MS_data/train_X.npy")
Y_1=np.load("drive/My Drive/MS_data/train_Y.npy")

## MODELLING and PARAMETERS

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Fri Dec 13 18:56:04 2019

@author: Krishna Chandra
"""
import tensorflow as tf
import keras
from keras.models import Model, load_model
from keras.layers import Input ,BatchNormalization , Activation 
from keras.layers.convolutional import Conv3D, UpSampling3D
from keras.layers.pooling import MaxPooling3D
from keras.layers.merge import concatenate


def Convolution(input_tensor,filters):
    
    x = Conv3D(filters=filters,kernel_size=(3, 3, 3),padding = 'same',strides=(1, 1, 1))(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 
    return x
  



def model(input_shape):
    
    inputs = Input((input_shape))
    
    conv_1 = Convolution(inputs,32)                                                                                         # 8,256,256,32
    
    maxp_1 = MaxPooling3D(pool_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (conv_1)                            # 4,128,128,32    
    conv_2 = Convolution(maxp_1,64)                                                                                         # 4,128,128,64
    maxp_2 = MaxPooling3D(pool_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (conv_2)                            # 2,64,64,64
    
    conv_3 = Convolution(maxp_2,128)                                                                                        # 2,64,64,128
    
    maxp_3 = MaxPooling3D(pool_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (conv_3)                            # 1,32,32,128 
    
    
    conv_4 = Convolution(maxp_3,256)                                                                                        # 1,32,32,256
   
    upsample_5 = UpSampling3D((2, 2, 2)) (conv_4)                                                                           # 2,64,64,256

    upsample_5 = concatenate([upsample_5, conv_3])                                                                          # 2,64,64,256+128
    
    conv_5 = Convolution(upsample_5,128)                                                                                    # 2,64,64,128
    upsample_6 = UpSampling3D((2, 2, 2)) (conv_5)                                                                           # 4,128,128,128                                                                                
    
    upsample_6 = concatenate([upsample_6, conv_2])                                                                          # 4,128,128,128+64
    
    conv_6 = Convolution(upsample_6,64)                                                                                     # 4,128,128,64        
    upsample_7 = UpSampling3D((2, 2, 2)) (conv_6)                                                                           # 8,256,256,64   
    
    upsample_7 = concatenate([upsample_7, conv_1])                                                                          # 8,256,256,64+32  

    conv_8 = Convolution(upsample_7,32)                                                                                     # 8,256,256,32
    
    
    outputs = Conv3D(1, (1, 1, 1), activation='sigmoid') (conv_8)
    
    model = Model(inputs=[inputs], outputs=[outputs]) 
    
    return model
     
    



Using TensorFlow backend.


In [0]:

from keras import backend as K
import numpy as np
import tensorflow as tf

# Computing Dice_Coefficient
def dice_coef(y_true, y_pred, smooth=1.0):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Computing Precision 
def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

# Computing Sensitivity      
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

# Computing Specificity
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

## TRAINING

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

#import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input ,BatchNormalization , Activation 
from tensorflow.keras.layers import Conv3D, UpSampling3D
from tensorflow.keras.layers import MaxPooling3D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

X_train , X_test, Y_train, Y_test = train_test_split(X_1, Y_1, test_size=0.15, random_state=32)


# Loding the modified U-net 
model = model(input_shape = (8,256,256,5))
model.summary()

checkpointer = ModelCheckpoint('drive/My Drive/MS_data/Modified_UNet_3D_Union_ckpt.h5', verbose=1)
callback_list=[checkpointer]

# Compiling the model
k_adam=Adam(lr=0.001)
model.compile(optimizer=k_adam, loss='binary_crossentropy', metrics=['accuracy',dice_coef,precision,sensitivity,specificity])
# Fitting the model over the data
history = model.fit(X_train,Y_train,batch_size=8,epochs=60,validation_split=0.20,verbose=1,initial_epoch=0,callbacks=callback_list)

# Saving the model
model.save('drive/My Drive/MS_data/Modified_UNet_3D_Union.h5')
history.history

# Evaluating the model on the training and testing data 
model.evaluate(x=X_train, y=Y_train, batch_size=8 , verbose=1, sample_weight=None, steps=None)
model.evaluate(x=X_test, y=Y_test, batch_size=8, verbose=1, sample_weight=None, steps=None)


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8, 256, 256, 0                                            
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, 8, 256, 256,  4352        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 8, 256, 256,  128         conv3d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 8, 256, 256,  0           batch_normalization[0][0]        
______________________________________________________________________________________________

[0.0005684037460014224,
 0.9998953938484192,
 0.998874306678772,
 0.9999828934669495,
 0.9994743466377258,
 0.9999971389770508]

## ANALYSIS

In [0]:
import numpy as np
import matplotlib.pyplot as plt

# Accuracy vs Epoch
def Accuracy_Graph(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    #plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.subplots_adjust(top=1.00, bottom=0.0, left=0.0, right=0.95, hspace=0.25,
                        wspace=0.35)
    plt.show()
    
# Dice Similarity Coefficient vs Epoch
def Dice_coefficient_Graph(history):

    plt.plot(history.history['dice_coef'])
    plt.plot(history.history['val_dice_coef'])
    #plt.title('Dice_Coefficient')
    plt.ylabel('Dice_Coefficient')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.subplots_adjust(top=1.00, bottom=0.0, left=0.0, right=0.95, hspace=0.25,
                        wspace=0.35)
    plt.show()
# Loss vs Epoch
def Loss_Graph(history):

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    #plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.subplots_adjust(top=1.00, bottom=0.0, left=0.0, right=0.95, hspace=0.25,
                        wspace=0.35)
    plt.show()
Accuracy_Graph(history)
Dice_coefficient_Graph(history)
Loss_Graph(history)


# 3D INTERSECTION MODEL

## Function Definitions



In [0]:
import tensorflow as tf
import numpy as np
import os
import nibabel as nib
import cv2 as cv
import matplotlib.pyplot as plt
    
def modality(Path,index):
    X = []
    X_per_paitent = []
    p=os.listdir(Path) 
    recs_in=[]
    counter=0
    counter_2=0
    #recs_segmented=[]
    for i in p[:14]:                                                                      # Loading all the folders in the given path
        q = os.listdir(os.path.join(Path,i))     

        x = nib.load(os.path.join(Path,i,q[index]))         
        f = x.get_fdata()
        f = np.asarray(f,'float32')
        
        ct=0
        recs_in.append(f.shape[2])
        #print(counter_2)
        counter_2+=1
        for j in range(f.shape[2]):                                                        # Processing the MRI Scan in the axial view
            _slice = cv.resize(f[:,:,j],(256,256),interpolation=cv.INTER_NEAREST)             # Resizing the slice to the shape(256,256)
            if(index not in [3,4,5,6,7,8,9] and np.sum(_slice) != 0 ): 
                if index==1:
                  ct+=1 
                  counter+=1 
                 # print(counter)                                      # To check whether the slice is null or not
              #  _slice = _slice / (np.max(_slice) + 0.00001)                               # Normalization
                  _slice = (_slice - np.mean(_slice) + 0.00001) / (np.std(_slice) + 0.00001)
                else:
                                              # To check whether the slice is null or not
              #  _slice = _slice / (np.max(_slice) + 0.00001)                               # Normalization
                  _slice = (_slice - np.mean(_slice) + 0.00001) / (np.std(_slice) + 0.00001) # Standardization
            elif(index in [3,4,5,6,7,8,9]):   # if index = 3, Then it is output mask and we don't normalize or standardize it 
                _slice = np.array(_slice)
                _slice[_slice > 0] = 1.0
                _slice[_slice < 0] = 0.0
            _slice = _slice.T
            _slice = _slice[:,:,np.newaxis]
            X.append(_slice)
       
   # X=np.array(X,dtype='float32')
    return X,recs_in

In [0]:
def remove_null_samples(X_Dp, X_Flair, X_Gado, X_T1, X_T2, Y_Manual,recs): 
     
    X=[]
    Y=[]
    counter=0
    counter_2=0
    mult=0;
    count=0
    rec=[]
    keep_count=[]
    keep=[]
    print(recs)
    r=np.array(recs,dtype='float32')
    print(np.sum(r))
    print(len(X_Dp))

    for i in range(len(X_Dp)):  
        if counter==(recs[mult]):
          print(counter)
          mult+=1
          rec.append(count)
          counter=0
          print(counter_2)
          count=0
        final_slice = np.concatenate((X_Dp[i],X_Flair[i],X_Gado[i],X_T1[i],X_T2[i]), axis = -1)
        if(np.sum(final_slice) != 0):        # checking whether the final slice is empty or not             
            X.append(final_slice)
            Y.append(Y_Manual[i])
            
            count+=1
        counter+=1
        counter_2+=1

    
    rec.append(count)
#   Converting the list into array  
    X=np.array(X,dtype='float32')
    Y=np.array(Y,dtype='float32')
    rec=np.array(rec,dtype='float32')
    
    return X,Y,rec


In [0]:
def store_data(X,Y,rec):
    np.save("drive/My Drive/MS_data/X_intersection_new.npy",X)
    np.save("drive/My Drive/MS_data/Y_intersection_new.npy",Y)
    np.save("drive/My Drive/MS_data/intersection_new_rec_after_removal.npy",rec)


In [0]:
import math
def intersection(Y_1,Y_2,Y_3,Y_4,Y_5,Y_6,Y_7):
  Y=[]
  sum2=[]
  flag=0
  #y=np.array()
  print("A")
  for i in range (len(Y_1)):
    #print(Y_1[i])
    
        f=np.concatenate((Y_1[i],Y_2[i],Y_3[i],Y_4[i],Y_5[i],Y_6[i],Y_7[i]),axis=-1)
        sum=np.sum(f,axis=2)
       # print(sum)
        
          #print(j)
        sum_1=np.divide(sum,7)
        sum_1=np.floor(sum_1)
        sum2.append(sum_1)

    #sum2=np.array(sum2,dtype='float32')
    
        
  return sum2
      

## Preprocessing Dataets

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

#import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input ,BatchNormalization , Activation 
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


#import dataPrepare as process
# Loading all the 5 different modalities of each MRI Scan of all 15 different patients and 1st rater Manual SegmentationX_Dp      =   modality(Path,0)
#import Modified_UNet 
#import plots
#import Metrics

# Setting the path
Path='drive/My Drive/Pre-processed'



# Loading all the 5 different modalities of each MRI Scan of all 15 different patients and 1st rater Manual Segmentation
X_Dp_t,rec      =   modality(Path,0)
X_Flair_t,rec_1   =   modality(Path,1)
X_Gado_t,rec    =   modality(Path,2)
X_T1_t,rec      =   modality(Path,10)
X_T2_t,rec      =   modality(Path,11)
rec=np.array(rec_1,dtype='float32')


In [0]:
np.save("drive/My Drive/MS_data/intersection_new_rec_before_removal.npy",rec)

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

#import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input ,BatchNormalization , Activation 
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


#import dataPrepare as process
# Loading all the 5 different modalities of each MRI Scan of all 15 different patients and 1st rater Manual SegmentationX_Dp      =   modality(Path,0)
#import Modified_UNet 
#import plots
#import Metrics

# Setting the path
Path='drive/My Drive/Pre-processed'
Y_1,rec  =   modality(Path,3)
Y_2,rec  =   modality(Path,4)
Y_3,rec  =   modality(Path,5)
Y_4,rec  =   modality(Path,6)
Y_5,rec  =   modality(Path,7)
Y_6,rec  =   modality(Path,8)
Y_7,rec  =   modality(Path,9)


In [0]:
Y_Manual=intersection(Y_1,Y_2,Y_3,Y_4,Y_5,Y_6,Y_7)

A


In [0]:
import numpy as np

np.save("drive/My Drive/MS_data/Y_manual_new.npy",Y_Manual)

In [0]:
Y_manual=list(Y_Manual)

In [0]:
Y_Manual=np.load("drive/My Drive/MS_data/Y_manual_new.npy")

In [0]:
X, Y,rec = remove_null_samples(X_Dp_t, X_Flair_t, X_Gado_t, X_T1_t, X_T2_t, Y_manual,rec_1)

In [0]:
store_data(X,Y,rec)

In [0]:
import numpy as np
Y=np.load("drive/My Drive/MS_data/Y_intersection_new.npy")

In [0]:
Y_Manual=list(Y)

In [0]:
Y_1=Y_Manual[:1401]
Y_2=Y_Manual[1401:]

In [0]:
p=[]
y=[]
z=[]
a=[]
n=1
for i in Y_1:
  print(n)
  for j in i:
    for k in j:
      #print(k)
      a.append(k)
     # print(a)
      z.append(a)
      a=[]
    #print(z)
    y.append(z)
    z=[]
  p.append(y)
  y=[]
  n+=1


#print(p)


In [0]:
temp=np.array(p)
np.save("drive/My Drive/MS_data/temp_1_new.npy",temp)

In [0]:
q=[]
y=[]
z=[]
a=[]
n=1401
for i in Y_2:
  print(n)
  for j in i:
    for k in j:
      #print(k)
      a.append(k)
     # print(a)
      z.append(a)
      a=[]
    #print(z)
    y.append(z)
    z=[]
  q.append(y)
  y=[]
  n+=1


In [0]:
temp2=np.array(q)
np.save("drive/My Drive/MS_data/temp_2_new.npy",temp2)

In [0]:
temp=np.load("drive/My Drive/MS_data/temp_1_new.npy")
p=list(temp)

In [0]:
temp2=np.load("drive/My Drive/MS_data/temp_2_new.npy")
q=list(temp2)

In [0]:
j=1401
for i in q:
  print(j)
  p.append(i)
  j+=1

In [0]:
np.save("drive/My Drive/MS_data/Y_Manual_2_new.npy",p)

In [0]:
P=np.load("drive/My Drive/MS_data/Y_Manual_2_new.npy")

In [0]:
print(np.min(P))
print(np.max(P))

0.0
1.0


In [0]:
Y=P

In [0]:
Y=list(Y)

In [0]:
dummy_y=[]
d_temp_1=[]
d_temp_2=[]
for i in range(len(Y[0])):
  d_temp_1=[]
  for j in range(len(Y[0][0])):
    d_temp_2=[]
    for k in range(len(Y[0][0][0])):
      d_temp_2.append(0.)
    d_temp_1.append(d_temp_2)
  dummy_y.append(d_temp_1)
Y_dummy=np.array(dummy_y, dtype='float32')




In [0]:
X=np.load("drive/My Drive/MS_data/X_intersection_new.npy")

In [0]:
X=list(X)

In [0]:
dummy_x=[]
d_temp_1=[]
d_temp_2=[]
for i in range(len(X[0])):
  d_temp_1=[]
  for j in range(len(X[0][0])):
    d_temp_2=[]
    for k in range(len(X[0][0][0])):
      d_temp_2.append(0.)
    d_temp_1.append(d_temp_2)
  dummy_x.append(d_temp_1)
X_dummy=np.array(dummy_x, dtype='float32')

In [0]:
import numpy as np

#Y=np.load("drive/My Drive/MS_data/Y_Manual_2_new.npy")
X=np.load("drive/My Drive/MS_data/X_union_new.npy")
#r=np.load("drive/My Drive/MS_data/union_new_rec_after_removal.npy")

In [0]:
X=list(X)
#Y=list(Y)

In [0]:
Y=Y.tolist()

In [0]:
extra=[0,1,6,4,6,0,5,0,4,3,1,3,3,0]

In [0]:
ext=np.array(extra,dtype='float32')

In [0]:
s=np.sum(ext)

In [0]:
r=np.load("drive/My Drive/MS_data/intersection_new_rec_after_removal.npy")

In [0]:
rec=list(r)

In [0]:
j=0
index=0
for i in extra:
  k=0
  
  print(i)
  index=index+rec[j]
  j+=1
  while(k<i):
    index=index+k
    print(index)
    index=int(index)
    Y.insert(index,Y_dummy)
    X.insert(index,X_dummy)

    k+=1

In [0]:
rec=list(r)
X=list(X)
Y=list(Y)

In [0]:
X=np.array(X,dtype='float32')
Y=np.array(Y,dtype='float32')

In [0]:
print(np.min(P))
print(np.max(P))

In [0]:
np.save("drive/My Drive/MS_data/Actual_X.npy",X)
np.save("drive/My Drive/MS_data/Actual_Y.npy",Y)

In [0]:
import numpy as np

Y=np.load("drive/My Drive/MS_data/Actual_X.npy")
X=np.load("drive/My Drive/MS_data/Actual_Y.npy")

In [0]:
X_1=[]
Y_1=[]


In [0]:
X=list(X)
Y=list(Y)

In [0]:
index_x_p=0
length=8
index_x=length
index_y_p=0
index_y=length
x_temp=[]
y_temp=[]




In [0]:
while index_x<=len(X) and index_y<=len(Y):
  x_temp=X[index_x_p:index_x]
  y_temp=Y[index_y_p:index_y]
  x_temp_1=np.array(x_temp,dtype='float32')
  y_temp_1=np.array(y_temp,dtype='float32')
  X_1.append(x_temp_1)
  Y_1.append(y_temp_1)
  index_y_p=index_y_p+length
  index_x_p=index_x_p+length
  index_x=index_x+length
  index_y=index_y+length


In [0]:
X_1=np.array(X_1,dtype='float32')
Y_1=np.array(Y_1,dtype='float32')


In [0]:
print(np.min(Y_1))
print(np.max(Y_1))

0.0
1.0


In [0]:
np.save("drive/My Drive/MS_data/train_X.npy",X_1)
np.save("drive/My Drive/MS_data/train_Y.npy",Y_1)

In [0]:
import numpy as np

X_1=np.load("drive/My Drive/MS_data/train_X.npy")
Y_1=np.load("drive/My Drive/MS_data/train_Y.npy")

## Modelling And Parameters

In [0]:

import tensorflow as tf
import keras
from keras.models import Model, load_model
from keras.layers import Input ,BatchNormalization , Activation 
from keras.layers.convolutional import Conv3D, UpSampling3D
from keras.layers.pooling import MaxPooling3D
from keras.layers.merge import concatenate


def Convolution(input_tensor,filters):
    
    x = Conv3D(filters=filters,kernel_size=(3, 3, 3),padding = 'same',strides=(1, 1, 1))(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 
    return x
  



def model(input_shape):
    
    inputs = Input((input_shape))
    
    conv_1 = Convolution(inputs,32)                                                                                         # 8,256,256,32
    
    maxp_1 = MaxPooling3D(pool_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (conv_1)                            # 4,128,128,32    
    conv_2 = Convolution(maxp_1,64)                                                                                         # 4,128,128,64
    maxp_2 = MaxPooling3D(pool_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (conv_2)                            # 2,64,64,64
    
    conv_3 = Convolution(maxp_2,128)                                                                                        # 2,64,64,128
    
    maxp_3 = MaxPooling3D(pool_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (conv_3)                            # 1,32,32,128 
    
    
    conv_4 = Convolution(maxp_3,256)                                                                                        # 1,32,32,256
   
    upsample_5 = UpSampling3D((2, 2, 2)) (conv_4)                                                                           # 2,64,64,256

    upsample_5 = concatenate([upsample_5, conv_3])                                                                          # 2,64,64,256+128
    
    conv_5 = Convolution(upsample_5,128)                                                                                    # 2,64,64,128
    upsample_6 = UpSampling3D((2, 2, 2)) (conv_5)                                                                           # 4,128,128,128                                                                                
    
    upsample_6 = concatenate([upsample_6, conv_2])                                                                          # 4,128,128,128+64
    
    conv_6 = Convolution(upsample_6,64)                                                                                     # 4,128,128,64        
    upsample_7 = UpSampling3D((2, 2, 2)) (conv_6)                                                                           # 8,256,256,64   
    
    upsample_7 = concatenate([upsample_7, conv_1])                                                                          # 8,256,256,64+32  

    conv_8 = Convolution(upsample_7,32)                                                                                     # 8,256,256,32
    
    
    outputs = Conv3D(1, (1, 1, 1), activation='sigmoid') (conv_8)
    
    model = Model(inputs=[inputs], outputs=[outputs]) 
    
    return model
     
    


In [0]:

from keras import backend as K
import numpy as np
import tensorflow as tf

# Computing Dice_Coefficient
def dice_coef(y_true, y_pred, smooth=1.0):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Computing Precision 
def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

# Computing Sensitivity      
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

# Computing Specificity
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

## TRAINING 

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

#import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input ,BatchNormalization , Activation 
from tensorflow.keras.layers import Conv3D, UpSampling3D
from tensorflow.keras.layers import MaxPooling3D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

X_train , X_test, Y_train, Y_test = train_test_split(X_1, Y_1, test_size=0.15, random_state=32)


# Loding the modified U-net 
model = model(input_shape = (8,256,256,5))
model.summary()

checkpointer = ModelCheckpoint('drive/My Drive/MS_data/Modified_UNet_3D_Intersection_ckpt.h5', verbose=1)
callback_list=[checkpointer]

# Compiling the model
k_adam=Adam(lr=0.001)
model.compile(optimizer=k_adam, loss='binary_crossentropy', metrics=['accuracy',dice_coef,precision,sensitivity,specificity])
# Fitting the model over the data
history = model.fit(X_train,Y_train,batch_size=8,epochs=60,validation_split=0.20,verbose=1,initial_epoch=0,callbacks=callback_list)

# Saving the model
model.save('drive/My Drive/MS_data/Modified_UNet_3D_Intersection.h5')
history.history

# Evaluating the model on the training and testing data 
model.evaluate(x=X_train, y=Y_train, batch_size=8 , verbose=1, sample_weight=None, steps=None)
model.evaluate(x=X_test, y=Y_test, batch_size=8, verbose=1, sample_weight=None, steps=None)


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 8, 256, 256, 0                                            
__________________________________________________________________________________________________
conv3d_8 (Conv3D)               (None, 8, 256, 256,  4352        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_7 (BatchNor (None, 8, 256, 256,  128         conv3d_8[0][0]                   
__________________________________________________________________________________________________
activation_7 (Activation)       (None, 8, 256, 256,  0           batch_normalization_7[0][0]      
____________________________________________________________________________________________

[0.0037747330497950315,
 0.998433530330658,
 0.5160220265388489,
 0.8746342062950134,
 0.5038745403289795,
 0.9998205900192261]

##  Analysis

In [0]:
import numpy as np
import matplotlib.pyplot as plt

# Accuracy vs Epoch
def Accuracy_Graph(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    #plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.subplots_adjust(top=1.00, bottom=0.0, left=0.0, right=0.95, hspace=0.25,
                        wspace=0.35)
    plt.show()
    
# Dice Similarity Coefficient vs Epoch
def Dice_coefficient_Graph(history):

    plt.plot(history.history['dice_coef'])
    plt.plot(history.history['val_dice_coef'])
    #plt.title('Dice_Coefficient')
    plt.ylabel('Dice_Coefficient')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.subplots_adjust(top=1.00, bottom=0.0, left=0.0, right=0.95, hspace=0.25,
                        wspace=0.35)
    plt.show()
# Loss vs Epoch
def Loss_Graph(history):

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    #plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.subplots_adjust(top=1.00, bottom=0.0, left=0.0, right=0.95, hspace=0.25,
                        wspace=0.35)
    plt.show()
Accuracy_Graph(history)
Dice_coefficient_Graph(history)
Loss_Graph(history)


# 3D MAJOR MODEL

## Function Definitions

In [0]:
import tensorflow as tf
import numpy as np
import os
import nibabel as nib
import cv2 as cv
import matplotlib.pyplot as plt
    
def modality(Path,index):
    X = []
    X_per_paitent = []
    p=os.listdir(Path) 
    recs_in=[]
    counter=0
    counter_2=0
    #recs_segmented=[]
    for i in p[:14]:                                                                      # Loading all the folders in the given path
        q = os.listdir(os.path.join(Path,i))     

        x = nib.load(os.path.join(Path,i,q[index]))         
        f = x.get_fdata()
        f = np.asarray(f,'float32')
        
        ct=0
        recs_in.append(f.shape[2])
        #print(counter_2)
        counter_2+=1
        for j in range(f.shape[2]):                                                        # Processing the MRI Scan in the axial view
            _slice = cv.resize(f[:,:,j],(256,256),interpolation=cv.INTER_NEAREST)             # Resizing the slice to the shape(256,256)
            if(index not in [3,4,5,6,7,8,9] and np.sum(_slice) != 0 ): 
                if index==1:
                  ct+=1 
                  counter+=1 
                 # print(counter)                                      # To check whether the slice is null or not
              #  _slice = _slice / (np.max(_slice) + 0.00001)                               # Normalization
                  _slice = (_slice - np.mean(_slice) + 0.00001) / (np.std(_slice) + 0.00001)
                else:
                                              # To check whether the slice is null or not
              #  _slice = _slice / (np.max(_slice) + 0.00001)                               # Normalization
                  _slice = (_slice - np.mean(_slice) + 0.00001) / (np.std(_slice) + 0.00001) # Standardization
            elif(index in [3,4,5,6,7,8,9]):   # if index = 3, Then it is output mask and we don't normalize or standardize it 
                _slice = np.array(_slice)
                _slice[_slice > 0] = 1.0
                _slice[_slice < 0] = 0.0
            _slice = _slice.T
            _slice = _slice[:,:,np.newaxis]
            X.append(_slice)
       
   # X=np.array(X,dtype='float32')
    return X,recs_in

In [0]:
def remove_null_samples(X_Dp, X_Flair, X_Gado, X_T1, X_T2, Y_Manual,recs): 
     
    X=[]
    Y=[]
    counter=0
    counter_2=0
    mult=0;
    count=0
    rec=[]
    keep_count=[]
    keep=[]
    print(recs)
    r=np.array(recs,dtype='float32')
    print(np.sum(r))
    print(len(X_Dp))

    for i in range(len(X_Dp)):  
        if counter==(recs[mult]):
          print(counter)
          mult+=1
          rec.append(count)
          counter=0
          print(counter_2)
          count=0
        final_slice = np.concatenate((X_Dp[i],X_Flair[i],X_Gado[i],X_T1[i],X_T2[i]), axis = -1)
        if(np.sum(final_slice) != 0):        # checking whether the final slice is empty or not             
            X.append(final_slice)
            Y.append(Y_Manual[i])
            
            count+=1
        counter+=1
        counter_2+=1

    
    rec.append(count)
#   Converting the list into array  
    X=np.array(X,dtype='float32')
    Y=np.array(Y,dtype='float32')
    rec=np.array(rec,dtype='float32')
    
    return X,Y,rec


In [0]:
def store_data(X,Y,rec):
    np.save("drive/My Drive/MS_data/X_major_new.npy",X)
    np.save("drive/My Drive/MS_data/Y_major_new.npy",Y)
    np.save("drive/My Drive/MS_data/major_new_rec_after_removal.npy",rec)


In [0]:
import math
def Major(Y_1,Y_2,Y_3,Y_4,Y_5,Y_6,Y_7):
  Y=[]
  sum2=[]
  flag=0
  #y=np.array()
  print("A")
  for i in range (len(Y_1)):
    #print(Y_1[i])
    
        f=np.concatenate((Y_1[i],Y_2[i],Y_3[i],Y_4[i],Y_5[i],Y_6[i],Y_7[i]),axis=-1)
        sum=np.sum(f,axis=2)
       # print(sum)
        
          #print(j)
        sum_1=np.divide(sum,4)
        sum_1=np.floor(sum_1)
        sum2.append(sum_1)

    #sum2=np.array(sum2,dtype='float32')
    
        
  return sum2
      

## Data Preprocessing 

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

#import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input ,BatchNormalization , Activation 
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


#import dataPrepare as process
# Loading all the 5 different modalities of each MRI Scan of all 15 different patients and 1st rater Manual SegmentationX_Dp      =   modality(Path,0)
#import Modified_UNet 
#import plots
#import Metrics

# Setting the path
Path='drive/My Drive/Pre-processed'



# Loading all the 5 different modalities of each MRI Scan of all 15 different patients and 1st rater Manual Segmentation
X_Dp_t,rec      =   modality(Path,0)
X_Flair_t,rec_1   =   modality(Path,1)
X_Gado_t,rec    =   modality(Path,2)
X_T1_t,rec      =   modality(Path,10)
X_T2_t,rec      =   modality(Path,11)
rec=np.array(rec_1,dtype='float32')


In [0]:
np.save("drive/My Drive/MS_data/major_new_rec_before_removal.npy",rec)

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

#import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input ,BatchNormalization , Activation 
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


#import dataPrepare as process
# Loading all the 5 different modalities of each MRI Scan of all 15 different patients and 1st rater Manual SegmentationX_Dp      =   modality(Path,0)
#import Modified_UNet 
#import plots
#import Metrics

# Setting the path
Path='drive/My Drive/Pre-processed'
Y_1,rec  =   modality(Path,3)
Y_2,rec  =   modality(Path,4)
Y_3,rec  =   modality(Path,5)
Y_4,rec  =   modality(Path,6)
Y_5,rec  =   modality(Path,7)
Y_6,rec  =   modality(Path,8)
Y_7,rec  =   modality(Path,9)


In [0]:
Y_Manual=Major(Y_1,Y_2,Y_3,Y_4,Y_5,Y_6,Y_7)

A


In [0]:
import numpy as np

np.save("drive/My Drive/MS_data/Y_manual_new.npy",Y_Manual)

In [0]:
Y_manual=list(Y_Manual)

In [0]:
Y_Manual=np.load("drive/My Drive/MS_data/Y_manual_new.npy")

In [0]:
X, Y,rec = remove_null_samples(X_Dp_t, X_Flair_t, X_Gado_t, X_T1_t, X_T2_t, Y_manual,rec_1)

In [0]:
store_data(X,Y,rec)

In [0]:
import numpy as np
Y=np.load("drive/My Drive/MS_data/Y_major_new.npy")

In [0]:
Y_Manual=list(Y)

In [0]:
Y_1=Y_Manual[:1401]
Y_2=Y_Manual[1401:]

In [0]:
p=[]
y=[]
z=[]
a=[]
n=1
for i in Y_1:
  print(n)
  for j in i:
    for k in j:
      #print(k)
      a.append(k)
     # print(a)
      z.append(a)
      a=[]
    #print(z)
    y.append(z)
    z=[]
  p.append(y)
  y=[]
  n+=1


#print(p)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [0]:
temp=np.array(p)
np.save("drive/My Drive/MS_data/temp_1_new.npy",temp)

In [0]:
q=[]
y=[]
z=[]
a=[]
n=1401
for i in Y_2:
  print(n)
  for j in i:
    for k in j:
      #print(k)
      a.append(k)
     # print(a)
      z.append(a)
      a=[]
    #print(z)
    y.append(z)
    z=[]
  q.append(y)
  y=[]
  n+=1


1401
1402
1403
1404
1405
1406
1407
1408
1409
1410
1411
1412
1413
1414
1415
1416
1417
1418
1419
1420
1421
1422
1423
1424
1425
1426
1427
1428
1429
1430
1431
1432
1433
1434
1435
1436
1437
1438
1439
1440
1441
1442
1443
1444
1445
1446
1447
1448
1449
1450
1451
1452
1453
1454
1455
1456
1457
1458
1459
1460
1461
1462
1463
1464
1465
1466
1467
1468
1469
1470
1471
1472
1473
1474
1475
1476
1477
1478
1479
1480
1481
1482
1483
1484
1485
1486
1487
1488
1489
1490
1491
1492
1493
1494
1495
1496
1497
1498
1499
1500
1501
1502
1503
1504
1505
1506
1507
1508
1509
1510
1511
1512
1513
1514
1515
1516
1517
1518
1519
1520
1521
1522
1523
1524
1525
1526
1527
1528
1529
1530
1531
1532
1533
1534
1535
1536
1537
1538
1539
1540
1541
1542
1543
1544
1545
1546
1547
1548
1549
1550
1551
1552
1553
1554
1555
1556
1557
1558
1559
1560
1561
1562
1563
1564
1565
1566
1567
1568
1569
1570
1571
1572
1573
1574
1575
1576
1577
1578
1579
1580
1581
1582
1583
1584
1585
1586
1587
1588
1589
1590
1591
1592
1593
1594
1595
1596
1597
1598
1599
1600


In [0]:
temp2=np.array(q)
np.save("drive/My Drive/MS_data/temp_2_new.npy",temp2)

In [0]:
temp=np.load("drive/My Drive/MS_data/temp_1_new.npy")
p=list(temp)

In [0]:
temp2=np.load("drive/My Drive/MS_data/temp_2_new.npy")
q=list(temp2)

In [0]:
j=1401
for i in q:
  print(j)
  p.append(i)
  j+=1

1401
1402
1403
1404
1405
1406
1407
1408
1409
1410
1411
1412
1413
1414
1415
1416
1417
1418
1419
1420
1421
1422
1423
1424
1425
1426
1427
1428
1429
1430
1431
1432
1433
1434
1435
1436
1437
1438
1439
1440
1441
1442
1443
1444
1445
1446
1447
1448
1449
1450
1451
1452
1453
1454
1455
1456
1457
1458
1459
1460
1461
1462
1463
1464
1465
1466
1467
1468
1469
1470
1471
1472
1473
1474
1475
1476
1477
1478
1479
1480
1481
1482
1483
1484
1485
1486
1487
1488
1489
1490
1491
1492
1493
1494
1495
1496
1497
1498
1499
1500
1501
1502
1503
1504
1505
1506
1507
1508
1509
1510
1511
1512
1513
1514
1515
1516
1517
1518
1519
1520
1521
1522
1523
1524
1525
1526
1527
1528
1529
1530
1531
1532
1533
1534
1535
1536
1537
1538
1539
1540
1541
1542
1543
1544
1545
1546
1547
1548
1549
1550
1551
1552
1553
1554
1555
1556
1557
1558
1559
1560
1561
1562
1563
1564
1565
1566
1567
1568
1569
1570
1571
1572
1573
1574
1575
1576
1577
1578
1579
1580
1581
1582
1583
1584
1585
1586
1587
1588
1589
1590
1591
1592
1593
1594
1595
1596
1597
1598
1599
1600


In [0]:
np.save("drive/My Drive/MS_data/Y_Manual_2_new.npy",p)

In [0]:
import numpy as np

P=np.load("drive/My Drive/MS_data/Y_Manual_2_new.npy")

In [0]:
print(np.min(P))
print(np.max(P))

0.0
1.0


In [0]:
Y=P

In [0]:
Y=list(Y)

In [0]:
dummy_y=[]
d_temp_1=[]
d_temp_2=[]
for i in range(len(Y[0])):
  d_temp_1=[]
  for j in range(len(Y[0][0])):
    d_temp_2=[]
    for k in range(len(Y[0][0][0])):
      d_temp_2.append(0.)
    d_temp_1.append(d_temp_2)
  dummy_y.append(d_temp_1)
Y_dummy=np.array(dummy_y, dtype='float32')




In [0]:
X=np.load("drive/My Drive/MS_data/X_major_new.npy")

In [0]:
X=list(X)

In [0]:
dummy_x=[]
d_temp_1=[]
d_temp_2=[]
for i in range(len(X[0])):
  d_temp_1=[]
  for j in range(len(X[0][0])):
    d_temp_2=[]
    for k in range(len(X[0][0][0])):
      d_temp_2.append(0.)
    d_temp_1.append(d_temp_2)
  dummy_x.append(d_temp_1)
X_dummy=np.array(dummy_x, dtype='float32')

In [0]:
import numpy as np

#Y=np.load("drive/My Drive/MS_data/Y_Manual_2_new.npy")
X=np.load("drive/My Drive/MS_data/X_union_new.npy")
#r=np.load("drive/My Drive/MS_data/union_new_rec_after_removal.npy")

In [0]:
X=list(X)
#Y=list(Y)

In [0]:
extra=[0,1,6,4,6,0,5,0,4,3,1,3,3,0]

In [0]:
ext=np.array(extra,dtype='float32')

In [0]:
s=np.sum(ext)

In [0]:
r=np.load("drive/My Drive/MS_data/major_new_rec_after_removal.npy")

In [0]:
rec=list(r)

In [0]:
j=0
index=0
for i in extra:
  k=0
  
  print(i)
  index=index+rec[j]
  j+=1
  while(k<i):
    index=index+k
    print(index)
    index=int(index)
    Y.insert(index,Y_dummy)
    X.insert(index,X_dummy)

    k+=1

0
1
279.0
6
561.0
562
564
567
571
576
4
844.0
845
847
850
6
988.0
989
991
994
998
1003
0
5
1590.0
1591
1593
1596
1600
0
4
2036.0
2037
2039
2042
3
2183.0
2184
2186
1
2377.0
3
2598.0
2599
2601
3
2782.0
2783
2785
0


In [0]:
rec=list(r)
X=list(X)
Y=list(Y)

In [0]:
X=np.array(X,dtype='float32')
Y=np.array(Y,dtype='float32')

In [0]:
np.save("drive/My Drive/MS_data/Actual_X.npy",X)
np.save("drive/My Drive/MS_data/Actual_Y.npy",Y)

In [0]:
import numpy as np

Y=np.load("drive/My Drive/MS_data/Actual_X.npy")
X=np.load("drive/My Drive/MS_data/Actual_Y.npy")

In [0]:
X_1=[]
Y_1=[]

In [0]:
X=list(X)
Y=list(Y)

In [0]:
index_x_p=0
length=8
index_x=length
index_y_p=0
index_y=length
x_temp=[]
y_temp=[]




In [0]:
while index_x<=len(X) and index_y<=len(Y):
  x_temp=X[index_x_p:index_x]
  y_temp=Y[index_y_p:index_y]
  x_temp_1=np.array(x_temp,dtype='float32')
  y_temp_1=np.array(y_temp,dtype='float32')
  X_1.append(x_temp_1)
  Y_1.append(y_temp_1)
  index_y_p=index_y_p+length
  index_x_p=index_x_p+length
  index_x=index_x+length
  index_y=index_y+length


In [0]:
X_1=np.array(X_1,dtype='float32')
Y_1=np.array(Y_1,dtype='float32')


In [0]:
print(np.min(Y_1))
print(np.max(Y_1))

0.0
1.0


In [0]:
np.save("drive/My Drive/MS_data/train_X.npy",X_1)
np.save("drive/My Drive/MS_data/train_Y.npy",Y_1)

In [0]:
import numpy as np

X_1=np.load("drive/My Drive/MS_data/train_X.npy")
Y_1=np.load("drive/My Drive/MS_data/train_Y.npy")

In [0]:
print(X_1.shape)

(372, 8, 256, 256, 5)


## Modelling and Parameter

In [0]:

import tensorflow as tf
import keras
from keras.models import Model, load_model
from keras.layers import Input ,BatchNormalization , Activation 
from keras.layers.convolutional import Conv3D, UpSampling3D
from keras.layers.pooling import MaxPooling3D
from keras.layers.merge import concatenate


def Convolution(input_tensor,filters):
    
    x = Conv3D(filters=filters,kernel_size=(3, 3, 3),padding = 'same',strides=(1, 1, 1))(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 
    return x
  



def model(input_shape):
    
    inputs = Input((input_shape))
    
    conv_1 = Convolution(inputs,32)                                                                                         # 8,256,256,32
    
    maxp_1 = MaxPooling3D(pool_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (conv_1)                            # 4,128,128,32    
    conv_2 = Convolution(maxp_1,64)                                                                                         # 4,128,128,64
    maxp_2 = MaxPooling3D(pool_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (conv_2)                            # 2,64,64,64
    
    conv_3 = Convolution(maxp_2,128)                                                                                        # 2,64,64,128
    
    maxp_3 = MaxPooling3D(pool_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same') (conv_3)                            # 1,32,32,128 
    
    
    conv_4 = Convolution(maxp_3,256)                                                                                        # 1,32,32,256
   
    upsample_5 = UpSampling3D((2, 2, 2)) (conv_4)                                                                           # 2,64,64,256

    upsample_5 = concatenate([upsample_5, conv_3])                                                                          # 2,64,64,256+128
    
    conv_5 = Convolution(upsample_5,128)                                                                                    # 2,64,64,128
    upsample_6 = UpSampling3D((2, 2, 2)) (conv_5)                                                                           # 4,128,128,128                                                                                
    
    upsample_6 = concatenate([upsample_6, conv_2])                                                                          # 4,128,128,128+64
    
    conv_6 = Convolution(upsample_6,64)                                                                                     # 4,128,128,64        
    upsample_7 = UpSampling3D((2, 2, 2)) (conv_6)                                                                           # 8,256,256,64   
    
    upsample_7 = concatenate([upsample_7, conv_1])                                                                          # 8,256,256,64+32  

    conv_8 = Convolution(upsample_7,32)                                                                                     # 8,256,256,32
    
    
    outputs = Conv3D(1, (1, 1, 1), activation='sigmoid') (conv_8)
    
    model = Model(inputs=[inputs], outputs=[outputs]) 
    
    return model
     
    


Using TensorFlow backend.


In [0]:

from keras import backend as K
import numpy as np
import tensorflow as tf

# Computing Dice_Coefficient
def dice_coef(y_true, y_pred, smooth=1.0):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Computing Precision 
def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

# Computing Sensitivity      
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

# Computing Specificity
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

## Training

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

#import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input ,BatchNormalization , Activation 
from tensorflow.keras.layers import Conv3D, UpSampling3D
from tensorflow.keras.layers import MaxPooling3D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

X_train , X_test, Y_train, Y_test = train_test_split(X_1, Y_1, test_size=0.15, random_state=32)


# Loding the modified U-net 
model = model(input_shape = (8,256,256,5))
model.summary()

checkpointer = ModelCheckpoint('drive/My Drive/MS_data/Modified_UNet_3D_Major_ckpt.h5', verbose=1)
callback_list=[checkpointer]

# Compiling the model
k_adam=Adam(lr=0.001)
model.compile(optimizer=k_adam, loss='binary_crossentropy', metrics=['accuracy',dice_coef,precision,sensitivity,specificity])
# Fitting the model over the data
history = model.fit(X_train,Y_train,batch_size=8,epochs=60,validation_split=0.20,verbose=1,initial_epoch=0,callbacks=callback_list)

# Saving the model
model.save('drive/My Drive/MS_data/Modified_UNet_3D_Major.h5')
history.history

# Evaluating the model on the training and testing data 
model.evaluate(x=X_train, y=Y_train, batch_size=8 , verbose=1, sample_weight=None, steps=None)
model.evaluate(x=X_test, y=Y_test, batch_size=8, verbose=1, sample_weight=None, steps=None)


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8, 256, 256, 0                                            
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, 8, 256, 256,  4352        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 8, 256, 256,  128         conv3d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 8, 256, 256,  0           batch_normalization[0][0]        
______________________________________________________________________________________________

[0.00780336232855916,
 0.9977254867553711,
 0.7346310019493103,
 0.8733747601509094,
 0.8037696480751038,
 0.9991928339004517]

In [0]:
print(X_train.shape)

(316, 8, 256, 256, 5)


## Analysis

In [0]:
import numpy as np
import matplotlib.pyplot as plt

# Accuracy vs Epoch
def Accuracy_Graph(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    #plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.subplots_adjust(top=1.00, bottom=0.0, left=0.0, right=0.95, hspace=0.25,
                        wspace=0.35)
    plt.show()
    
# Dice Similarity Coefficient vs Epoch
def Dice_coefficient_Graph(history):

    plt.plot(history.history['dice_coef'])
    plt.plot(history.history['val_dice_coef'])
    #plt.title('Dice_Coefficient')
    plt.ylabel('Dice_Coefficient')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.subplots_adjust(top=1.00, bottom=0.0, left=0.0, right=0.95, hspace=0.25,
                        wspace=0.35)
    plt.show()
# Loss vs Epoch
def Loss_Graph(history):

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    #plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.subplots_adjust(top=1.00, bottom=0.0, left=0.0, right=0.95, hspace=0.25,
                        wspace=0.35)
    plt.show()
Accuracy_Graph(history)
Dice_coefficient_Graph(history)
Loss_Graph(history)


NameError: ignored